In [4]:
import database_handler
db_session = database_handler.create_connection(None)

# # Create a database session
# database_handler.create_connection("your_database_connection_parameters")

# # Your ETL logic and operations go here
# ...


In [15]:
import database_handler 

database_handler.read_data_as_dataframe('csv', "C:\Users\User\Downloads\Fifa_world_cup_matches.csv")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (921342391.py, line 3)

In [ ]:
import data_handler

film_dataframe = data_handler.returns_query_as_dataframe(db_session, 'SELECT * FROM dvd_rental.film')


: 

In [ ]:
import lookups
import pandas_handler
def return_dataframe_info(dataframe):
    info_dictionary = dict()
    info_dictionary['dataframe_length'] = len(dataframe)
    info_dictionary['total_count_of_table_id'] = int(dataframe.describe().iloc[0][0])
    return info_dictionary

def return_dataframe_subset_from_to_location(dataframe, from_loc, to_loc):
    return dataframe.loc[from_loc,to_loc]
def return_filtered_dataframe(dataframe, filter_field, filter_value):
    return dataframe[dataframe[filter_field.value] > filter_value]

: 

In [ ]:
import database_handler
database_handler.create_connection(None)
    
